## Jointure et GroupBy

In [1]:
import pandas as pd

In [2]:
#Exemple de jointure

# DataFrame 1 : Clients
df_clients = pd.DataFrame({
    'client_id': [1, 2, 3, 4],
    'nom': ['Alice', 'Bob', 'Charlie', 'David'],
    'ville': ['Paris', 'Lyon', 'Marseille', 'Toulouse']
})

# DataFrame 2 : Commandes
df_commandes = pd.DataFrame({
    'commande_id': [101, 102, 103, 104],
    'client_id': [1, 2, 3, 1],
    'montant': [250, 150, 200, 300]
})

display(df_clients)

display(df_commandes)

,client_id,nom,ville
0,1,Alice,Paris
1,2,Bob,Lyon
2,3,Charlie,Marseille
3,4,David,Toulouse


,commande_id,client_id,montant
0,101,1,250
1,102,2,150
2,103,3,200
3,104,1,300


In [3]:
# Effectuer une jointure (INNER JOIN) sur la colonne 'client_id'
df_jointure = df_clients.merge(df_commandes, on='client_id')
df_jointure

,client_id,nom,ville,commande_id,montant
0,1,Alice,Paris,101,250
1,1,Alice,Paris,104,300
2,2,Bob,Lyon,102,150
3,3,Charlie,Marseille,103,200


In [4]:
#Création d'un dataframe `df_commandes_colonnes_differentes`
#pour renommer la colonne "client_id" et illustrer les jointures avec
#de colonnes de noms différents

df_commandes_colonnes_differentes = df_commandes.copy()
df_commandes_colonnes_differentes.rename(columns={"client_id": "personne id"}, inplace=True)
print("Liste des commandes:")
display(df_commandes_colonnes_differentes)
print("Liste des clients:")
display(df_clients)

Liste des commandes:


,commande_id,personne id,montant
0,101,1,250
1,102,2,150
2,103,3,200
3,104,1,300


Liste des clients:


,client_id,nom,ville
0,1,Alice,Paris
1,2,Bob,Lyon
2,3,Charlie,Marseille
3,4,David,Toulouse


In [5]:
# Effectuer une jointure (INNER JOIN) avec des noms des colonnes différents
df_clients.merge(df_commandes_colonnes_differentes, left_on='client_id', right_on='personne id')

,client_id,nom,ville,commande_id,personne id,montant
0,1,Alice,Paris,101,1,250
1,1,Alice,Paris,104,1,300
2,2,Bob,Lyon,102,2,150
3,3,Charlie,Marseille,103,3,200


In [6]:
# Effectuer une jointure (LEFT JOIN)
df_jointure2 = df_clients.merge(df_commandes, on='client_id', how='left')
df_jointure2

,client_id,nom,ville,commande_id,montant
0,1,Alice,Paris,101.0,250.0
1,1,Alice,Paris,104.0,300.0
2,2,Bob,Lyon,102.0,150.0
3,3,Charlie,Marseille,103.0,200.0
4,4,David,Toulouse,NaN,NaN


In [7]:
# Repérer les données qui se trouvent uniquement dans le DataFrame de gauche

df_jointure_left_sans_union = df_clients.merge(df_commandes, on='client_id', how='left')
df_jointure_left_sans_union[df_jointure_left_sans_union["commande_id"].isnull()]

,client_id,nom,ville,commande_id,montant
4,4,David,Toulouse,NaN,NaN


### Exemple de commande GROUP BY
Utilisation de la fonction **groupby**

In [8]:
#Exemple de group by

# Grouper par 'client_id' et calculer le montant total dépensé
df_groupby = df_jointure.groupby('client_id')['montant'].sum().reset_index()

# Afficher le DataFrame résultant
display(df_groupby)

,client_id,montant
0,1,550
1,2,150
2,3,200


In [9]:
pd.DataFrame(df_jointure.groupby('client_id')['montant'].sum())

,montant
client_id,
1,550
2,150
3,200


In [24]:
# Grouper par 'client_id' et calculer le montant total dépensé
df_groupby2 = df_jointure.groupby(['client_id']).agg(somme_total=("montant", "sum"))

# Afficher le DataFrame résultant
display(df_groupby2)

,somme_total
client_id,
1,550
2,150
3,200


In [25]:
df_groupby2["somme_total"].sum()

900

### Repérer des données se trouvant dans un dataframe mais pas dans l'autre
Utilisation de la fonction merge avec un merge 'left' et indicator=True pour repérer uniquement les données présentes dans le premier dataframe

In [13]:
import pandas as pd

# df1 : Liste des personnes dans df1
df1 = pd.DataFrame({
    'nom': ['Dupont', 'Martin', 'Durand', 'Bernard'],
    'prenom': ['Alice', 'Bob', 'Charlie', 'David']
})

# df2 : Liste des personnes dans df2
df2 = pd.DataFrame({
    'nom': ['Martin', 'Bernard', 'Lefevre'],
    'prenom': ['Bob', 'David', 'Emma']
})

print("DataFrame df1:")
display(df1)
print("\nDataFrame df2:")
display(df2)

DataFrame df1:


,nom,prenom
0,Dupont,Alice
1,Martin,Bob
2,Durand,Charlie
3,Bernard,David



DataFrame df2:


,nom,prenom
0,Martin,Bob
1,Bernard,David
2,Lefevre,Emma


In [14]:
# Effectuer une jointure externe gauche (left join)
# indicator=True : Ajoute une colonne _merge indiquant l'origine de chaque ligne.
df_diff = pd.merge(df1, df2, on=['nom', 'prenom'], how='left', indicator=True)
df_diff

,nom,prenom,_merge
0,Dupont,Alice,left_only
1,Martin,Bob,both
2,Durand,Charlie,left_only
3,Bernard,David,both


In [15]:
# Sélectionner uniquement les lignes où '_merge' vaut 'left_only'
df_diff2 = df_diff[df_diff['_merge'] == 'left_only']
df_diff2

,nom,prenom,_merge
0,Dupont,Alice,left_only
2,Durand,Charlie,left_only


In [16]:
print("\nPersonnes présentes dans df1 mais pas dans df2:")
df_diff2[['nom','prenom']]


Personnes présentes dans df1 mais pas dans df2:


,nom,prenom
0,Dupont,Alice
2,Durand,Charlie


In [17]:
df2[['nom', 'prenom']].to_dict(orient='list')

{'nom': ['Martin', 'Bernard', 'Lefevre'], 'prenom': ['Bob', 'David', 'Emma']}

In [27]:
# Autre méthode (utilisation de la fonction isin)
df1[['nom', 'prenom']].isin(df2[['nom', 'prenom']].to_dict(orient='list'))

,nom,prenom
0,False,False
1,True,True
2,False,False
3,True,True


In [28]:
~df1[['nom', 'prenom']].isin(df2[['nom', 'prenom']].to_dict(orient='list')).all(axis="columns")

0     True
1    False
2     True
3    False
dtype: bool

In [29]:
mask = ~df1[['nom', 'prenom']].isin(df2[['nom', 'prenom']].to_dict(orient='list')).all(axis="columns")
mask

0     True
1    False
2     True
3    False
dtype: bool

In [30]:
df1[mask]

,nom,prenom
0,Dupont,Alice
2,Durand,Charlie
